In [ ]:
# LLM , 사전학습 모델을 
#  나만의 데이터를 다룰수 있도록 하는 방법은

# 방법 1. 전이학습 (그중의 하나 Fine tunning (미세조정))
# 방벙 2. RAG : Retrieval Augmented Generation (검색증강생성)

# RAG
Retrieval Augmented Generation (검색증강생성)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough

In [ ]:
# 사전학습된 모델은 이미 많은 데이터를 통해 학습한 상태이긴 하나..
# 개인 DB 나 회사내 문서 와 같이 'private 한 데이터' 들에는 접근할수 없다
# 그래서 RAG 를 사용한다!

In [ ]:
# 1. Retrieval 단계
# private 으로부터 제공된 data 를 사용하거나 탐색함으로써
# language model 의 능력을 더 '확장(augment)'

# 2. Augmented Generation
# Model 로 하여금 '우리가 보낸 문서 data 만'을 가지고 답변하도록 할수도 있다.
# (경우에 따라, 우리 문서가 더 최신 data 일수도 있기 때문이다)
# 이를 통해 Model 이 과거에 학습한 data 를 참조하지 않게도 할수 있다.

In [ ]:
# RAG 는 특정 라이브러리나 프레임워크 이름이 아니라
# 위와 같은 작업을 하는 '기법'을 일반적으로 통칭하는 용어

# RAG 를 수행하는 방법은 굉~장히 많고 다양.